# 🚀 Proyecto Integrador 1: Chatbot de Consulta de Datos con RAG

**Objetivo**: construir un chatbot empresarial que permita consultar datos usando lenguaje natural, combinando RAG (documentación) y NL2SQL (queries).

## Alcance del Proyecto

- **Duración**: 4-6 horas
- **Dificultad**: Alta
- **Stack**: OpenAI, ChromaDB, LangChain, SQLite/PostgreSQL, Streamlit

## Funcionalidades

1. ✅ Indexar documentación de esquemas y métricas
2. ✅ Responder preguntas sobre estructura de datos (RAG)
3. ✅ Ejecutar queries en lenguaje natural (NL2SQL)
4. ✅ Visualizar resultados en tablas/gráficos
5. ✅ Historial de conversación
6. ✅ Validación de seguridad

## Parte 1: Setup del proyecto

In [ ]:
# pip install openai chromadb langchain streamlit pandas plotly sqlalchemy
import os
import sqlite3
import pandas as pd
from openai import OpenAI
import chromadb
from chromadb.config import Settings

# Configuración
os.environ['OPENAI_API_KEY'] = 'tu-api-key'  # Reemplazar
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

print('✅ Setup completo')

## Parte 2: Base de datos de ejemplo

In [ ]:
# Crear BD SQLite con datos de ejemplo
conn = sqlite3.connect('empresa.db')
cursor = conn.cursor()

# Tabla ventas
cursor.execute('''
CREATE TABLE IF NOT EXISTS ventas (
    venta_id INTEGER PRIMARY KEY,
    fecha DATE,
    producto TEXT,
    categoria TEXT,
    cantidad INTEGER,
    precio_unitario REAL,
    total REAL,
    region TEXT
)
''')

# Datos de ejemplo
ventas_data = [
    (1, '2024-01-15', 'Laptop Pro', 'Electrónica', 2, 1200, 2400, 'Norte'),
    (2, '2024-01-16', 'Mouse Wireless', 'Accesorios', 10, 25, 250, 'Sur'),
    (3, '2024-01-17', 'Teclado Mecánico', 'Accesorios', 5, 80, 400, 'Norte'),
    (4, '2024-01-18', 'Monitor 27"', 'Electrónica', 3, 350, 1050, 'Este'),
    (5, '2024-01-19', 'Laptop Pro', 'Electrónica', 1, 1200, 1200, 'Oeste'),
    (6, '2024-01-20', 'Webcam HD', 'Accesorios', 8, 60, 480, 'Norte')
]

cursor.executemany('INSERT OR REPLACE INTO ventas VALUES (?,?,?,?,?,?,?,?)', ventas_data)
conn.commit()

# Verificar
df_ventas = pd.read_sql_query('SELECT * FROM ventas LIMIT 3', conn)
print('✅ Base de datos creada\n')
print(df_ventas)

## Parte 3: Indexar documentación (RAG)

In [ ]:
# ChromaDB para RAG
chroma_client = chromadb.PersistentClient(path='./chatbot_db')
collection = chroma_client.get_or_create_collection(name='data_docs')

def get_embedding(text: str):
    resp = client.embeddings.create(
        model='text-embedding-ada-002',
        input=text
    )
    return resp.data[0].embedding

# Documentación a indexar
docs = [
    {
        'id': 'tabla_ventas',
        'text': '''
Tabla: ventas
Descripción: Registro de todas las transacciones de venta de productos.
Columnas:
- venta_id: ID único de la venta
- fecha: Fecha de la transacción
- producto: Nombre del producto vendido
- categoria: Categoría (Electrónica, Accesorios)
- cantidad: Unidades vendidas
- precio_unitario: Precio por unidad en USD
- total: Monto total (cantidad * precio_unitario)
- region: Región geográfica (Norte, Sur, Este, Oeste)
        '''
    },
    {
        'id': 'metrica_revenue',
        'text': '''
Métrica: Revenue Total
Definición: Suma del campo 'total' de todas las ventas.
Cálculo: SELECT SUM(total) FROM ventas
Uso: Dashboard ejecutivo, reportes mensuales
        '''
    },
    {
        'id': 'producto_top',
        'text': '''
Análisis: Top Productos
Query: SELECT producto, SUM(cantidad) as unidades, SUM(total) as revenue 
       FROM ventas GROUP BY producto ORDER BY revenue DESC
Insight: Identifica productos más rentables
        '''
    }
]

# Indexar
for doc in docs:
    collection.add(
        ids=[doc['id']],
        documents=[doc['text']],
        embeddings=[get_embedding(doc['text'])]
    )

print(f'✅ {len(docs)} documentos indexados')

## Parte 4: Sistema RAG

In [ ]:
def rag_search(question: str, top_k: int = 2):
    """Busca contexto relevante."""
    query_emb = get_embedding(question)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=top_k
    )
    return '\n\n'.join(results['documents'][0])

def rag_answer(question: str):
    """Responde usando RAG."""
    context = rag_search(question)
    
    prompt = f'''
Eres un asistente de datos experto. Responde basándote SOLO en el contexto.

Contexto:
{context}

Pregunta: {question}

Respuesta:
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role': 'user', 'content': prompt}],
        temperature=0.1
    )
    
    return resp.choices[0].message.content.strip()

# Test
q1 = '¿Qué columnas tiene la tabla ventas?'
print(f'❓ {q1}')
print(f'✅ {rag_answer(q1)}\n')

q2 = '¿Cómo se calcula el revenue total?'
print(f'❓ {q2}')
print(f'✅ {rag_answer(q2)}')

## Parte 5: Sistema NL2SQL

In [ ]:
def get_schema():
    """Obtiene esquema de la BD."""
    return '''
Tabla: ventas
Columnas:
- venta_id INTEGER
- fecha DATE
- producto TEXT
- categoria TEXT
- cantidad INTEGER
- precio_unitario REAL
- total REAL
- region TEXT
'''

def is_safe_query(query: str):
    """Valida seguridad."""
    dangerous = ['INSERT', 'UPDATE', 'DELETE', 'DROP', 'ALTER', 'CREATE']
    return not any(kw in query.upper() for kw in dangerous)

def nl_to_sql(question: str):
    """Convierte pregunta a SQL."""
    schema = get_schema()
    
    prompt = f'''
Esquema de base de datos:
{schema}

Convierte esta pregunta a SQL (SQLite):
{question}

Reglas:
- Solo SELECT
- Devuelve SOLO el SQL, sin explicaciones

SQL:
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role': 'user', 'content': prompt}],
        temperature=0
    )
    
    return resp.choices[0].message.content.strip().replace('```sql','').replace('```','').strip()

def execute_nl_query(question: str):
    """Ejecuta query desde lenguaje natural."""
    try:
        sql = nl_to_sql(question)
        
        if not is_safe_query(sql):
            return {'error': 'Query no segura detectada'}
        
        df = pd.read_sql_query(sql, conn)
        
        return {
            'sql': sql,
            'data': df,
            'rows': len(df)
        }
    except Exception as e:
        return {'error': str(e)}

# Test
q3 = '¿Cuántas ventas hubo en total?'
result = execute_nl_query(q3)
print(f'❓ {q3}')
print(f'SQL generado: {result["sql"]}')
print(f'Resultado:\n{result["data"]}')

## Parte 6: Chatbot unificado

In [ ]:
def classify_intent(question: str):
    """Clasifica la intención de la pregunta."""
    prompt = f'''
Clasifica esta pregunta en UNA categoría:
- SCHEMA: pregunta sobre estructura de datos, definiciones
- QUERY: requiere ejecutar una consulta SQL

Pregunta: {question}

Responde solo con: SCHEMA o QUERY
'''
    
    resp = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{'role': 'user', 'content': prompt}],
        temperature=0
    )
    
    return resp.choices[0].message.content.strip()

def chatbot(question: str):
    """Chatbot inteligente que elige RAG o NL2SQL."""
    intent = classify_intent(question)
    
    if intent == 'SCHEMA':
        return {
            'type': 'RAG',
            'answer': rag_answer(question)
        }
    else:  # QUERY
        result = execute_nl_query(question)
        if 'error' in result:
            return {'type': 'ERROR', 'answer': result['error']}
        return {
            'type': 'SQL',
            'sql': result['sql'],
            'data': result['data'],
            'rows': result['rows']
        }

# Tests
preguntas = [
    '¿Qué información tiene la tabla ventas?',
    '¿Cuál fue el total de ventas por región?',
    '¿Cómo se define el revenue?',
    'Top 3 productos por ingresos'
]

for q in preguntas:
    print(f'\n❓ {q}')
    response = chatbot(q)
    print(f'Tipo: {response["type"]}')
    if response['type'] == 'SQL':
        print(f'SQL: {response["sql"]}')
        print(response['data'])
    else:
        print(response['answer'])

## Parte 7: Interfaz Streamlit

In [ ]:
# Guardar como app.py y ejecutar: streamlit run app.py

streamlit_code = '''
import streamlit as st
import pandas as pd
import plotly.express as px
from chatbot import chatbot  # Importar función

st.set_page_config(page_title='Data Chatbot', page_icon='🤖', layout='wide')

st.title('🤖 Chatbot de Consulta de Datos')
st.markdown('Pregunta sobre tus datos en lenguaje natural')

# Historial en session_state
if 'history' not in st.session_state:
    st.session_state.history = []

# Input
question = st.text_input('Tu pregunta:', placeholder='Ej: ¿Cuáles son las ventas totales por categoría?')

if st.button('Consultar') and question:
    with st.spinner('Procesando...'):
        response = chatbot(question)
        st.session_state.history.append({'q': question, 'r': response})
        
        if response['type'] == 'SQL':
            st.success(f"SQL ejecutado: `{response['sql']}`")
            st.dataframe(response['data'])
            
            # Visualización automática si es numérico
            if len(response['data'].columns) == 2:
                fig = px.bar(response['data'], x=response['data'].columns[0], y=response['data'].columns[1])
                st.plotly_chart(fig)
        else:
            st.info(response['answer'])

# Historial
if st.session_state.history:
    st.markdown('---')
    st.subheader('📜 Historial')
    for item in reversed(st.session_state.history[-5:]):
        st.markdown(f"**Q:** {item['q']}")
        if item['r']['type'] == 'SQL':
            st.code(item['r']['sql'])
        st.markdown('---')
'''

with open('app.py', 'w') as f:
    f.write(streamlit_code)

print('✅ App Streamlit guardada en app.py')
print('Ejecuta: streamlit run app.py')

## Parte 8: Mejoras y extensiones

### Siguientes pasos

1. **Caché**: implementa Redis para queries frecuentes
2. **Feedback loop**: permite al usuario marcar respuestas correctas/incorrectas
3. **Multi-tabla**: soporta JOINs automáticos
4. **Exportación**: permite descargar resultados en CSV/Excel
5. **Auditoría**: loggea todas las queries ejecutadas
6. **Permisos**: implementa autenticación y control de acceso
7. **Visualizaciones**: genera gráficos automáticos según el tipo de datos
8. **Sugerencias**: recomienda preguntas frecuentes

## Evaluación

**Criterios**:

- ✅ Sistema RAG funcional (20%)
- ✅ NL2SQL con validación de seguridad (25%)
- ✅ Clasificación correcta de intención (15%)
- ✅ Interfaz funcional (20%)
- ✅ Manejo de errores (10%)
- ✅ Código limpio y documentado (10%)